In [1]:
from ortools.linear_solver import pywraplp
import numpy as np



In [2]:
# data = np.array([['1', 9, 8, 0],
#             ['2', 7, 6, 2],
#             ['3', 6, 2, 4]])

data = np.array([['1', 8, 7],
               ['2', 6, 3]])

In [3]:
def Max_Weights(data):
    # Setting up necessary and useful variables for rest of function
    p = [] # Prices vector, turned into a matrix into a vector
    rows, columns = data.shape
    solver = pywraplp.Solver('Maximum Weights',
                               pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    
    # Objective: maximize ∑∑v*p for all agents values for courses v 
    # and probabilities of recieving courses p 
    objective = solver.Objective()
    for i in range(rows):
        agent_s = data[i][0]
        for j in range (1, columns):
            p.append(solver.NumVar(0.0, 1.0, agent_s))
            objective.SetCoefficient(p[i*(columns-1)+j-1], int(data[i][j]))
    objective.SetMaximization()
    
    

    # Setting constraints
    row_consts = []
    col_consts = []



    # Setting row constraints (Sum of all rows of p is 1)
    for i in range(rows):
        coeffs = np.zeros(len(p)) # done to avoid if branch in for loop when setting coefficients
        row_consts.append(solver.Constraint(1.0, 1.0)) # do these bounds work?????????????
        j=i*(columns - 1)
        while j < (i+1)*(columns - 1):
            coeffs[j] = 1
            j += 1 
        for k in range(len(p)):
            row_consts[i].SetCoefficient(p[k], coeffs[k])

    # Setting column constraints (Sum of all columns of p is 1)
    for i in range(columns - 1): 
        coeffs = np.zeros(len(p)) # done to avoid if branch in for loop when setting coefficients
        col_consts.append(solver.Constraint(1.0, 1.0)) # still unsure of bounds
        j = i
        while j < len(p):
            coeffs[j] = 1
            j += columns - 1 # assuming data format will be the same
        for k in range(len(p)):
            col_consts[i].SetCoefficient(p[k], coeffs[k])


    # Display results
    status = solver.Solve()
    mat = np.zeros(len(p))
    mat.shape = (rows, (columns - 1))
    for i in range(rows):
        for j in range(columns-1):
            mat[i][j] = p[i*(columns-1)+j].solution_value()
    return mat

In [4]:
print(Max_Weights(data))

[[0. 1.]
 [1. 0.]]
